In [1]:
import json
import pandas as pd
import hvplot.pandas
import requests
import numpy as np
from config import openweather_key

sales_data_path = "/Users/matthewflanagan/Desktop/Data Analytics Work/Project 1/Project_1/file.csv"


sales_data = pd.read_csv(sales_data_path)

sales_data.head()


,Unnamed: 0,CustomerID,Gender,Location,Tenure_Months,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,...,Avg_Price,Delivery_Charges,Coupon_Status,GST,Date,Offline_Spend,Online_Spend,Month,Coupon_Code,Discount_pct
0,0,17850.0,M,Chicago,12.0,16679.0,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.5,Used,0.1,1/1/2019,4500.0,2424.5,1,ELEC10,10.0
1,1,17850.0,M,Chicago,12.0,16680.0,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.5,Used,0.1,1/1/2019,4500.0,2424.5,1,ELEC10,10.0
2,2,17850.0,M,Chicago,12.0,16696.0,2019-01-01,GGOENEBQ078999,Nest Cam Outdoor Security Camera - USA,Nest-USA,...,122.77,6.5,Not Used,0.1,1/1/2019,4500.0,2424.5,1,ELEC10,10.0
3,3,17850.0,M,Chicago,12.0,16699.0,2019-01-01,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,...,81.50,6.5,Clicked,0.1,1/1/2019,4500.0,2424.5,1,ELEC10,10.0
4,4,17850.0,M,Chicago,12.0,16700.0,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.5,Clicked,0.1,1/1/2019,4500.0,2424.5,1,ELEC10,10.0


In [2]:

#dropping redundant and un-needed columns
sales_data_cleaned = sales_data.drop(['Unnamed: 0', 'Tenure_Months', 'Transaction_Date', 'GST', 'Offline_Spend', 'Online_Spend', 'Month'], axis=1)


#dropping rows that have incomplete data
sales_data_cleaned = sales_data_cleaned.dropna()


#converting the date to unix time format for the api request & and adding it to a column
sales_data_cleaned['Date'] = pd.to_datetime(sales_data_cleaned['Date'])

sales_data_cleaned["Unix Date"] = sales_data_cleaned['Date'].apply(lambda x: int(x.timestamp()))


sales_data_cleaned.head()

,CustomerID,Gender,Location,Transaction_ID,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Date,Coupon_Code,Discount_pct,Unix Date
0,17850.0,M,Chicago,16679.0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1.0,153.71,6.5,Used,2019-01-01,ELEC10,10.0,1546300800
1,17850.0,M,Chicago,16680.0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1.0,153.71,6.5,Used,2019-01-01,ELEC10,10.0,1546300800
2,17850.0,M,Chicago,16696.0,GGOENEBQ078999,Nest Cam Outdoor Security Camera - USA,Nest-USA,2.0,122.77,6.5,Not Used,2019-01-01,ELEC10,10.0,1546300800
3,17850.0,M,Chicago,16699.0,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,1.0,81.50,6.5,Clicked,2019-01-01,ELEC10,10.0,1546300800
4,17850.0,M,Chicago,16700.0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1.0,153.71,6.5,Clicked,2019-01-01,ELEC10,10.0,1546300800


In [3]:
#getting 100 random rows for each city
np.random.seed(42)

def random_rows(group):
    return group.sample(min(len(group), 100))

sales_data_random_cleaned = sales_data_cleaned.groupby('Location', group_keys=False).apply(random_rows)


#hard cording the city coordinates
city_coordinates = {
    "Chicago": (41.8781, -87.6298),
    "California": (36.7783, -119.4179),
    "New Jersey": (40.0583, -74.4057),
    "New York": (40.7128, -74.0060),
    "Washington DC": (38.8951, -77.0364),
}

#adding latitude and longitude columns
sales_data_random_cleaned["Lat"] = sales_data_random_cleaned["Location"].map(city_coordinates).apply(lambda x: x[0])
sales_data_random_cleaned["Lon"] = sales_data_random_cleaned["Location"].map(city_coordinates).apply(lambda x: x[1])


#adding blank weather conditions column
sales_data_random_cleaned["Weather Conditions"] = ""

sales_data_random_cleaned.head()

,CustomerID,Gender,Location,Transaction_ID,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Date,Coupon_Code,Discount_pct,Unix Date,Lat,Lon,Weather Conditions
31125,14911.0,F,California,19784.0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1.0,149.00,6.50,Used,2019-02-06,ELEC20,20.0,1549411200,36.7783,-119.4179,
18033,17596.0,F,California,22388.0,GGOEGDHC018299,Google 22 oz Water Bottle,Drinkware,32.0,2.99,6.50,Clicked,2019-03-08,EXTRA30,30.0,1552003200,36.7783,-119.4179,
44004,14911.0,F,California,32456.0,GGOEGFKQ020799,Google Doodle Decal,Office,1.0,2.39,49.73,Clicked,2019-07-12,OFF10,10.0,1562889600,36.7783,-119.4179,
1050,12433.0,M,California,17529.0,GGOENEBB078899,Nest Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,6.50,Clicked,2019-01-10,ELEC10,10.0,1547078400,36.7783,-119.4179,
20023,15456.0,M,California,30075.0,GGOEGFKA022299,Keyboard DOT Sticker,Office,2.0,1.50,6.00,Clicked,2019-06-11,OFF30,30.0,1560211200,36.7783,-119.4179,


In [9]:
#BE CAREFUL RUNNING API REQUEST AS WE ONLY GET 1000 PER DAY
#THIS DATA SET CONTAINS 500 PER RUN!!!
#takes ~5min to run

url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"
units = "imperial"


#setting the parameters for the api request
for index, row in sales_data_random_cleaned.iterrows():
    latitude = row["Lat"]
    longitude = row["Lon"]
    date = row["Unix Date"]

    params = {
    "lat": latitude,
    "lon": longitude,
    "dt": date,
    "apiKey":openweather_key,
    "units": units
}


    response = requests.get(url, params=params)

    response = response.json()

  
 #getting the weather description for each row and putting it in the weather conditions column
    try:
        sales_data_random_cleaned.loc[index, "Weather Conditions"] = response["data"][0]["weather"][0]["description"]
        
    except KeyError as e:
        print("No Weather Data Found")


sales_data_random_cleaned.head()


,CustomerID,Gender,Location,Transaction_ID,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Date,Coupon_Code,Discount_pct,Unix Date,Lat,Lon,Weather Conditions
31125,14911.0,F,California,19784.0,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1.0,149.00,6.50,Used,2019-02-06,ELEC20,20.0,1549411200,36.7783,-119.4179,light rain
18033,17596.0,F,California,22388.0,GGOEGDHC018299,Google 22 oz Water Bottle,Drinkware,32.0,2.99,6.50,Clicked,2019-03-08,EXTRA30,30.0,1552003200,36.7783,-119.4179,light rain
44004,14911.0,F,California,32456.0,GGOEGFKQ020799,Google Doodle Decal,Office,1.0,2.39,49.73,Clicked,2019-07-12,OFF10,10.0,1562889600,36.7783,-119.4179,clear sky
1050,12433.0,M,California,17529.0,GGOENEBB078899,Nest Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,6.50,Clicked,2019-01-10,ELEC10,10.0,1547078400,36.7783,-119.4179,moderate rain
20023,15456.0,M,California,30075.0,GGOEGFKA022299,Keyboard DOT Sticker,Office,2.0,1.50,6.00,Clicked,2019-06-11,OFF30,30.0,1560211200,36.7783,-119.4179,overcast clouds
